In [56]:
import os
import numpy as np
from PIL import Image
from decimal import Decimal, getcontext
from skimage.metrics import peak_signal_noise_ratio as psnr
import time
import heapq
import matplotlib.pyplot as plt


Nesta etapa inicial, as imagens utilizadas no experimento são carregadas e preparadas para o processo de compressão. As imagens são lidas a partir do disco utilizando a biblioteca Pillow, convertidas para escala de cinza e redimensionadas para um tamanho fixo de 255×255 pixels. Essa padronização é necessária para garantir uniformidade durante a análise estatística e a aplicação dos algoritmos de compressão.

Após o carregamento, cada imagem é representada como uma matriz NumPy de valores inteiros de 8 bits, correspondentes aos níveis de cinza entre 0 e 255. Em seguida, é gerado o histograma de símbolos de cada imagem, contabilizando a frequência de ocorrência de cada nível de intensidade. A partir desses histogramas, calcula-se a entropia de Shannon, que fornece uma estimativa teórica do limite inferior da compressão sem perdas.

In [57]:
#1. Seleção e Preparação das imagens para compressão:

# Caminho local onde as imagens de teste foram armazenadas.
# As imagens utilizadas neste projeto foram previamente baixadas manualmente
# e salvas neste diretório específico. Esse caminho pode (e deve) ser alterado
# conforme o ambiente de execução, sistema operacional ou método de obtenção
# das imagens (download automático, outro diretório, outro computador, etc.).
path = r"C:\Users\user\programação\.idea"

imgs = load_imagens(img_names, path)
hists = hist_simbolos(imgs)

img_names = ["kodim23t.jpg",
       "kodim22t.jpg",
       "kodim21t.jpg",
       "kodim20t.jpg",
       "kodim19t.jpg",
       "kodim17t.jpg",
       "kodim13t.jpg",
       "kodim06t.jpg",
       "kodim05t.jpg",
       "kodim02t.jpg"]
#Carregando imagens em escala de cinza e matriz uint8
def load_imagens(img_names, path, size=(255, 255)):
    imgs = []
    for filenames in img_names:
        img = Image.open(os.path.join(path, filenames)).convert("L")
        #imagens precisa de mesmo tamanho para não ocorrer erro no "histogram2d"
        img = img.resize(size)
        imgs.append(np.array(img, dtype = np.uint8))
    return imgs

#Gerando histograma de símbolos para as imagens
def hist_simbolos(imgs):
    hists = []
    for img in imgs:
        pixels = img.flatten()
        hist = np.bincount(pixels, minlength=256)
        hists.append(hist)
    return hists

#Calculando entropia das imagens
def calcular_entropia(hists):
    entropias = []
    for hist in hists:
        prob = hist / hist.sum()
        prob_nonzero = prob[prob > 0]
        H = -np.sum(prob_nonzero * np.log2(prob_nonzero))
        entropias.append(H)
    return entropias


Nesta seção são implementados os algoritmos de compressão sem perdas utilizados no trabalho: a codificação aritmética e a codificação de Huffman.

A codificação aritmética baseia-se na atribuição de intervalos cumulativos proporcionais à probabilidade de ocorrência dos símbolos. A sequência completa de dados é representada por um único número real dentro de um intervalo progressivamente refinado. Para isso, são construídas tabelas de probabilidade a partir das frequências dos símbolos observados na imagem.

In [58]:
#2. Implementar os Algoritmos de Compressão: Codificação aritmética
hist = hists[0]

frequency_table = {
    i: int(hist[i])
    for i in range(256)
    if hist[i] > 0
}

#Codificação Aritmética
class ArithmeticEncoding:

    #Construtor da codificação AE
    def __init__(self, frequency_table):
        self.probability_table = self.get_probability_table(frequency_table)

#Divide a frequência de cada símbolo pela soma de todas as frequências na tabela
def get_probability_table(self, frequency_table):
    total_frequency = sum(frequency_table.values())
    probability_table = {}
    for key in sorted(frequency_table.keys()):
        probability_table[key] = frequency_table[key] / total_frequency
    return probability_table

#Codificador
def encode(self, msg, probability_table):
    encoder = []
    stage_min = Decimal(0.0)
    stage_max = Decimal(1.0)
    for msg_term_idx in range(len(msg)):
        stage_probs = self.process_stage(probability_table, stage_min, stage_max)
        msg_term = msg[msg_term_idx]
        stage_min = stage_probs[msg_term][0]
        stage_max = stage_probs[msg_term][1]
        encoder.append(stage_probs)
    stage_probs = self.process_stage(probability_table, stage_min, stage_max)
    encoder.append(stage_probs)
    encoded_msg = self.get_encoded_value(encoder)
    return encoder, encoded_msg

#Decodificador
def decode(self, encoded_msg, msg_length, probability_table):
    decoder = []
    decoded_msg = []
    stage_min = Decimal(0.0)
    stage_max = Decimal(1.0)
    for idx in range(msg_length):
        stage_probs = self.process_stage(probability_table, stage_min, stage_max)
        for msg_term in sorted(stage_probs.keys()):
            value = stage_probs[msg_term]
            if encoded_msg >= value[0] and encoded_msg <= value[1]:
                break
        decoded_msg.append(msg_term)
        stage_min = stage_probs[msg_term][0]
        stage_max = stage_probs[msg_term][1]
        decoder.append(stage_probs)
    stage_probs = self.process_stage(probability_table, stage_min, stage_max)
    decoder.append(stage_probs)
    return decoder, decoded_msg

A codificação de Huffman, por sua vez, utiliza um método de codificação por comprimento variável baseado na construção de uma árvore binária ótima. Símbolos mais frequentes recebem códigos binários menores, enquanto símbolos menos frequentes recebem códigos maiores, garantindo a propriedade de prefixo e a minimização do comprimento médio do código.

In [59]:
#2. Implementar os Algoritmos de Compressão: Huffman

#Codificação de Huffman
class HuffmanNode:

    #Construtor da codifição de Huffman
    def __init__(self, symbol=None, freq=0):
        self.symbol = symbol
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

#Construção da arvode de Huffman
def build_huffman_tree(frequency_table):
    heap = []
    for symbol, freq in frequency_table.items():
        heapq.heappush(heap, HuffmanNode(symbol, freq))
    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(freq=node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2
        heapq.heappush(heap, merged)
    return heap[0]

#Gerador da arvore de Huffman
def generate_huffman_codes(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}
    if node.symbol is not None:
        codebook[node.symbol] = prefix
        return codebook
    generate_huffman_codes(node.left, prefix + "0", codebook)
    generate_huffman_codes(node.right, prefix + "1", codebook)
    return codebook

#Huffman codificação
def huffman_encode(img, codebook):
    flat = img.flatten()
    encoded_bits = "".join(codebook[pixel] for pixel in flat)
    return encoded_bits

#Huffman decodificação
def huffman_decode(encoded_bits, root, shape):
    decoded = []
    node = root
    for bit in encoded_bits:
        node = node.left if bit == "0" else node.right
        if node.symbol is not None:
            decoded.append(node.symbol)
            node = root
    return np.array(decoded, dtype=np.uint8).reshape(shape)


Ambos os algoritmos operam diretamente sobre os valores dos pixels, considerados como símbolos de uma fonte discreta.

O pipeline geral de compressão segue uma sequência estruturada de etapas, aplicada individualmente a cada imagem do conjunto de dados. Inicialmente, a imagem é recebida como uma matriz bidimensional de pixels em escala de cinza. Em seguida, essa matriz é transformada em um vetor unidimensional, permitindo o tratamento da imagem como uma sequência linear de símbolos.

A partir desse vetor, calcula-se a distribuição de probabilidade dos símbolos, que é utilizada para a construção do codificador de Huffman. O vetor de pixels é então codificado, gerando um bitstream comprimido. Posteriormente, esse bitstream é decodificado utilizando a mesma árvore de Huffman, permitindo a reconstrução da imagem original.

Por se tratar de um método de compressão sem perdas, a imagem reconstruída é idêntica à imagem original, o que é verificado ao final do processo.

In [60]:
#3. Pipeline Geral de Compressão
imgs = load_imagens(img_names, path)

for idx, img_array in enumerate(imgs):
    img_name = img_names[idx]

    #"Entrada"
    original_shape = img_array.shape

    #"Flattening"
    pixels_1d = img_array.flatten()

    #"Cálculo da distribuição de probabilidade"
    hist = np.bincount(pixels_1d, minlength=256)
    frequency_table = {
        symbol: int(freq)
        for symbol, freq in enumerate(hist)
        if freq > 0
    }

    #"Construção do codificador"
    root = build_huffman_tree(frequency_table)
    huffman_codes = generate_huffman_codes(root)

    #"Codificação do vetor de pixels"
    bitstream = huffman_encode(img_array, huffman_codes)

    #"Geração do bitstream comprimido"
    compressed_size_bits = len(bitstream)

    #"Decodificação reversa" - "Reconstrução da imagem"
    reconstructed_img = huffman_decode(bitstream, root, original_shape)

A análise dos resultados é realizada por meio de métricas quantitativas que avaliam a eficiência e a qualidade do processo de compressão. A taxa de compressão é calculada comparando o número de bits necessários para representar a imagem original com o número de bits gerados após a codificação.

A qualidade da imagem reconstruída é avaliada por meio do PSNR (Peak Signal-to-Noise Ratio). Como os métodos de compressão implementados são sem perdas, o erro quadrático médio entre a imagem original e a reconstruída é nulo, resultando em um PSNR teoricamente infinito.

Além disso, o tempo de processamento é medido para avaliar o custo computacional da compressão. Para fins de comparação, também é considerada a compressão JPEG, utilizando as funções nativas da biblioteca Pillow.

In [61]:
#4. Análise e Comparação:

#Calcular a Taxa de Compressão
def calculate_file_size(file_path):
    """Calcula o tamanho do arquivo em bytes."""
    return os.path.getsize(file_path)

def calculate_compression_ratio_array(original_img, bitstream):
    original_bits = original_img.size * 8
    compressed_bits = len(bitstream)
    return original_bits / compressed_bits

#Avaliar a Qualidade da Imagem
def calculate_psnr_array(original_img, reconstructed_img):
    mse = np.mean((original_img.astype(np.float64) - reconstructed_img.astype(np.float64)) ** 2)
    if mse == 0:
        return float("inf")
    max_pixel = 255.0
    return 10 * log10((max_pixel ** 2) / mse)

#Medir o Tempo de Processamento
def measure_time(function, *args):
    start = time.time()
    result = function(*args)
    end = time.time()
    return result, end - start

#Testes
def compress_image(input_path, output_path, quality):
    img = Image.open(input_path)
    img.save(output_path, 'JPEG', quality=quality)

for idx, img_array in enumerate(imgs):
    img_name = img_names[idx]

    original_shape = img_array.shape
    pixels_1d = img_array.flatten()

    hist = np.bincount(pixels_1d, minlength=256)
    frequency_table = {
        symbol: int(freq)
        for symbol, freq in enumerate(hist)
        if freq > 0
    }

    # Tempo de compressão
    (bitstream, root), comp_time = measure_time(
        lambda img: (
            huffman_encode(img, generate_huffman_codes(build_huffman_tree(frequency_table))),
            build_huffman_tree(frequency_table)
        ),
        img_array
    )

    reconstructed_img = huffman_decode(bitstream, root, original_shape)

    compression_ratio = calculate_compression_ratio_array(img_array, bitstream)
    psnr_value = calculate_psnr_array(img_array, reconstructed_img)

    print(f"Imagem: {img_name}")
    print(f"Taxa de Compressão: {compression_ratio:.2f}")
    print(f"PSNR: {psnr_value:.2f} dB")
    print(f"Tempo de Compressão: {comp_time:.4f} s\n")

Imagem: kodim23t.jpg
Taxa de Compressão: 1.24
PSNR: inf dB
Tempo de Compressão: 0.0047 s

Imagem: kodim22t.jpg
Taxa de Compressão: 1.25
PSNR: inf dB
Tempo de Compressão: 0.0047 s

Imagem: kodim21t.jpg
Taxa de Compressão: 1.27
PSNR: inf dB
Tempo de Compressão: 0.0088 s

Imagem: kodim20t.jpg
Taxa de Compressão: 1.35
PSNR: inf dB
Tempo de Compressão: 0.0049 s

Imagem: kodim19t.jpg
Taxa de Compressão: 1.24
PSNR: inf dB
Tempo de Compressão: 0.0049 s

Imagem: kodim17t.jpg
Taxa de Compressão: 1.22
PSNR: inf dB
Tempo de Compressão: 0.0048 s

Imagem: kodim13t.jpg
Taxa de Compressão: 1.22
PSNR: inf dB
Tempo de Compressão: 0.0046 s

Imagem: kodim06t.jpg
Taxa de Compressão: 1.22
PSNR: inf dB
Tempo de Compressão: 0.0061 s

Imagem: kodim05t.jpg
Taxa de Compressão: 1.23
PSNR: inf dB
Tempo de Compressão: 0.0047 s

Imagem: kodim02t.jpg
Taxa de Compressão: 1.49
PSNR: inf dB
Tempo de Compressão: 0.0048 s



Os resultados obtidos demonstram que a codificação de Huffman é capaz de reduzir significativamente o tamanho dos dados, aproximando-se do limite teórico imposto pela entropia da fonte. Observa-se que imagens com maior redundância apresentam taxas de compressão mais elevadas.

Em comparação com o método JPEG, a compressão por Huffman preserva integralmente a informação da imagem, ao custo de taxas de compressão geralmente menores. Essa diferença é esperada, uma vez que o JPEG utiliza técnicas com perdas baseadas em transformadas e quantização.